In [41]:
# %matplotlib inline
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
from mpl_toolkits.axes_grid1 import make_axes_locatable

from qick import *
from qick.helpers import gauss

import logging
logger = logging.getLogger('qick.qick_asm')
logger.setLevel(logging.ERROR)

from tqdm import tqdm_notebook as tqdm

import time
import os
expt_path=os.getcwd()+'/data'
import sys
import scipy as sp
import json

import qutip as qt
import qutip.visualization as qplt

from slab.instruments import InstrumentManager
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 16})
# style.use('default')
style.use('S:\Connie\prx.mplstyle')

np.set_printoptions(threshold=sys.maxsize)

Could not import QickInstrument (will only work if running on rfsoc). If running on rfsoc, try install/update qick package
Warning failed to import AWG81180A
Warning failed to import Tek5014
Warning failed to import Tek70001
Warning failed to import M8195A


c:\ProgramData\Anaconda3\lib\site-packages\IPython\qt.py:12: ShimWarning: The `IPython.qt` package has been deprecated since IPython 4.0. You should import from qtconsole instead.
  warn("The `IPython.qt` package has been deprecated since IPython 4.0. "


Could not load InstrumentManagerWindow
Could not load labbrick
Could not load Autonics TM4 Driver
Could not load Oxford Trition driver


In [3]:
sys.path.append(os.getcwd()+'/../../qutip_sims')
from QSwitch import QSwitch
from PulseSequence import PulseSequence

In [4]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

# Tomo analysis

In [5]:
from TomoAnalysis import TomoAnalysis
tomo_analysis = TomoAnalysis(nb_qubits=3)

# Getting ideal rho

Define psi basis

In [6]:
# 2 cardinal points on Bloch sphere for each measurement axis
# psiZ = [qt.basis(3,0), qt.basis(3,1)]
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

psis = dict() # psis in the ZZZ basis
for i, label_numeric in enumerate(tomo_analysis.calib_order_numeric):
    psis.update({label_numeric:tomo_analysis.psi_basis['ZZZ'][i]})
id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))
id2q = qt.tensor(qt.qeye(2), qt.qeye(2))

In [7]:
def name_to_state_2q(init_state): # in format |QA>|QB>
    Qa, Qb, _ = init_state.split('>')
    Qa = Qa[1:]
    Qb = Qb[1:]
    psi_name_dict = {
        '0':psiZ[0],
        '1':psiZ[1],
        '0+1':(psiZ[0]+psiZ[1]).unit(),
        '0-1':(psiZ[0]-psiZ[1]).unit(), 
        '0+i':(psiZ[0]+1j*psiZ[1]).unit(), 
        '0-i':(psiZ[0]-1j*psiZ[1]).unit(), 
    }
    try:
        psiA = psi_name_dict[Qa]
    except:
        print(f'QA state {Qa} is not an allowed cardinal state')
    try:
        psiB = psi_name_dict[Qb]
    except:
        print(f'QB state {Qb} is not an allowed cardinal state')
    return qt.tensor(psiA, psiB)

    
def get_qram_qSLR_state_from_rho(init_rho_SI, post_select=False, post_select_state=None):
    """
    init_rho_SI should be a np array
    """
    # in order |00>, |01>, |10>, |11> for Q0 (switch), Q1 (input)
    SI_to_SLR = [psis['000'], -1j*psis['010'], psis['100'], -1j*psis['101']]

    assert np.shape(init_rho_SI) == id2q.shape
    rho_final_SLR = 0*qt.ket2dm(psis['000'])
    # print('constructing final state on |switch, out1, out2>')
    for i in range(len(SI_to_SLR)):
        slr_ket_i = SI_to_SLR[i]
        for j in range(len(SI_to_SLR)):
            slr_ket_j = SI_to_SLR[j]
            mat_el = init_rho_SI[i, j]
            rho_final_SLR += mat_el * slr_ket_i * slr_ket_j.dag()
    return rho_final_SLR.unit()

# Run experiment and analysis

In [8]:
"""Reopen saved data"""
def prev_data(expt_path, filename=None, verbose=True):
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    if verbose: print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        if verbose: print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [9]:
data_path = 'S:\\QRAM\\qram_4QR2\\'
# expt_path=os.path.join(os.getcwd(), 'data', 'data_240617')
expt_path = os.path.join(data_path, 'data', 'data_250119')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_250119


In [10]:
# config_file = 'config_zcu216.yml'
# config_file = 'config_q3diamond.yml'
# config_file = 'config_q3diamond_full688_reset.yml'
config_file = 'config_q3diamond_full688and638_reset.yml'
# config_path = os.path.join(os.getcwd(), config_file)

# config_file = 'config.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)

print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


In [11]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [12]:
import Pyro4.util

## Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [13]:
# ns_address = '10.108.30.56' # Spilker Fermium
ns_address = '10.108.30.75' # Spilker Vibranium
im = InstrumentManager(ns_address=ns_address)
print(im.keys())

dict_keys(['Pyro.NameServer', 'Qick_QRAM'])


In [14]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)
# print(soc.description())
# print(soc.dump_cfg())

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

In [ ]:
lo1 = im[yaml_cfg.aliases.readout_LO]
lo1.open_device()
print(lo1.get_id())

# TURN OFF
lo1.set_standby(True)
lo1.set_output_state(False)

# # TURN ON
# lo1.set_standby(False)
# lo1.set_output_state(True)
# lo_freq = float(yaml_cfg.hw.lo.readout.frequency)
# lo_power = float(yaml_cfg.hw.lo.readout.power)
# lo1.set_frequency(lo_freq) # Set LO frequency
# print('set freq to', lo_freq)
# lo1.set_power(lo_power) # Set LO power
# print('set power to', lo_power)

## Run experiment and acquire data

### Test 3Q tomo

In [15]:
save_data = True
saved_files = []

In [ ]:
tomo_qubits = [0, 2, 3] # which qubit/resonator we want to probe

In [ ]:
tomoExpt = meas.TestStateTomo3QExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"test_3q_state_tomo_{tomo_qubits[0]}{tomo_qubits[1]}{tomo_qubits[2]}",
    config_file=config_path,
)

tomoExpt.cfg.expt = dict(
    reps=20000, # reps per measurement basis
    singleshot_reps=30000, # reps for single shot calib
    tomo_qubits=tomo_qubits,
)

print(tomoExpt.cfg)
import Pyro4.util
try:
    tomoExpt.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

if save_data: saved_files.append(tomoExpt.save_data())

In [ ]:
tomo = tomoExpt
# tomo = qramExpt

# data, attrs = prev_data(expt_path, '00062_qram_protocol.h5')
# pulse_dict = attrs['pulse_dict']
# assert np.all(np.array(attrs['meas_order']) == meas_order)
# assert np.all(np.array(attrs['calib_order']) == calib_order)

print(meas_order)
print(calib_order)

data = tomo.data
pulse_dict = tomo.pulse_dict
assert np.all(np.array(tomo.meas_order) == tomo_analysis.meas_order)
assert np.all(np.array(tomo.calib_order) == tomo_analysis.calib_order)

n_tomo_raw = np.array(data['counts_tomo'])
n_calib = np.array(data['counts_calib'])

np.set_printoptions(suppress=True)
print('Tomography counts')
print(n_tomo_raw)
print('Confusion matrix counts')
print(n_calib)
print('Confusion matrix corrected tomo matrix will be')
print(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib))
# print('Negative counts corrected tomo matrix will be')
# print(tomo_analysis.fix_neg_counts(correct_readout_err(n_tomo_raw, n_calib)))

### Qram experiment

In [536]:
tomo_qubits = [0, 2, 3] # which qubit/resonators to do tomo on
# tomo_qubits = [0, 1, 2] # which qubit/resonators to do tomo on

In [537]:
save_data = False
saved_files = []

In [585]:
# init_state = '|0>|0+i>' # Q0/Q1 (switch, input)
# init_state = '|1>|0+1>' # Q0/Q1 (switch, input)
# init_state = '|0+1>|1>' # Q0/Q1 (switch, input)
init_state = '|0+1>|0+1>' # Q0/Q1 (switch, input)

# init_state = 'Q2Q0_|1>|0>'
# init_state = '|0>|1>' # Q0/Q1 (switch, input)
# init_state = 'Q2Q0_|1>|0>'

# play_pulses = [0] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
play_pulses = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [1, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [2, 4] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

# cool_qubits = [0, 1]
cool_qubits = None
readout_cool = False

full_mux_expt = True
resonator_reset = [0, 2, 3]

use_IQ_pulse = False

use_robust_pulses = True

sync_between_swaps = False
sync_between_efs = False

In [ ]:
qramExpt = meas.QramProtocol3QTomoExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)

# span = 2.161196963262976 # total time [us]
# span = 1.1659462011134218 # total time [us]
span = np.inf
npts = 1
# span = 1.8 # total time [us]
# npts = 120
# span = 0.8 # total time [us]
# npts = 10
if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))

qramExpt.cfg.expt = dict(
    # start=0, # start protocol time [us],
    # step=span/npts, # time step [us], 
    # expts=npts+1, # number of different time experiments - added 1 to include 0 step and full span

    start=span,
    step=0,
    expts=1, # number of different time experiments

    reps=1000000, # number of reps per tomo/time step,
    # reps=10000, # number of reps per tomo/time step,
    # reps=1000,
    # singleshot_reps=1, # reps per state for singleshot calibration
    # singleshot_reps=10000, # reps per state for singleshot calibration
    # singleshot_reps=1000, # reps per state for singleshot calibration
    singleshot_reps=10,
    post_process='scale', # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
    tomo_3q=True, # flag to perform tomo on last time step
    tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
    init_state=init_state,
    play_pulses=play_pulses, # [2, 1, 4, 3]
    sync_between_swaps=sync_between_swaps,
    sync_between_efs=sync_between_efs,
    
    cool_qubits=cool_qubits,
    
    use_IQ_pulse=use_IQ_pulse,
    plot_IQ=False,
    
    readout_cool=readout_cool,
    n_init_readout=1,
    n_trig=1,
    init_read_wait_us=2,
    
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    
    use_robust_pulses=use_robust_pulses,
    
    # meas_order=['XXX'],
)
if tomo_qubits == [0, 2, 3]:
    qramExpt.cfg.device.readout.gain[1] = 1e-4


# print(qramExpt.cfg)

import Pyro4.util
try:
    qramExpt.acquire(debug=False, progress=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# tomo = qramExpt
# data = tomo.data
# n_tomo_raw = np.array(data['counts_tomo'])
# n_calib = np.array(data['counts_calib'])

# n_tomo_raw_rounds.append(n_tomo_raw)
# n_calib_rounds.append(n_calib)
# n_tomo_rounds.append(fix_neg_counts(correct_readout_err(n_tomo_raw, n_calib)))


In [ ]:
saved_files = [qramExpt.save_data()]

#### post processing counts

In [361]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

In [ ]:
tomo = qramExpt
data = tomo.data
cfg = tomo.cfg
pulse_dict = qramExpt.pulse_dict
init_state = qramExpt.cfg.expt.init_state

n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)

print('calib order', tomo_analysis.calib_order)

print('Raw tomography counts')
print(n_tomo_raw)
print('Confusion matrix counts')
print(n_calib)
print('Confusion matrix corrected tomo matrix will be')
print(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib))
# print('Negative counts corrected tomo matrix will be')
# print(tomo_analysis.fix_neg_counts(correct_readout_err(n_tomo_raw, n_calib))) 

In [ ]:
filename = saved_files[-1]

### Import optimal control pulse

In [ ]:
pulse_filepath = os.path.join(os.getcwd(), '240528_pulse_sequence_pp.npz')
pulse_params_dict = dict() # open file
with np.load(pulse_filepath) as npzfile:
    for key in npzfile.keys():
        pulse_params_dict.update({key:npzfile[key]})

print(pulse_params_dict.keys())
IQ_qubits = [0, 1]
print('IQ_qubits', IQ_qubits)

times = pulse_params_dict['times']
I_0 = pulse_params_dict['I_0']
Q_0 = pulse_params_dict['Q_0']
I_1 = pulse_params_dict['I_1']
Q_1 = pulse_params_dict['Q_1']
plt.figure()
plt.plot(times*1e6, I_0*1e-6, label='$I_0$')
plt.plot(times*1e6, Q_0*1e-6, label='$Q_0$')
plt.plot(times*1e6, I_1*1e-6, label='$I_1$')
plt.plot(times*1e6, Q_1*1e-6, label='$Q_1$')
plt.legend()
plt.xlabel('Times [us]')
plt.ylabel('Amplitude [MHz]')
plt.show()

I_values_MHz = np.array([I_0, I_1])*1e-6
Q_values_MHz = np.array([Q_0, Q_1])*1e-6
times_us = times*1e6

## Loop over delays on generators

In [ ]:
init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']

protocol_play = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

sync_between_swaps = True
sync_between_efs = True

tomo_qubits = [0, 2, 3] # which qubit/resonators to do tomo on
save_data = True
saved_files = []
# cool_qubits = [0, 1]
cool_qubits = None

readout_cool = False
full_mux_expt = True
resonator_reset = [0, 2, 3]

use_IQ_pulse = False # state transfer IQ pulses for the state prep
use_robust_pulses = True # use robust pulses for state prep + tomo

delay_sweep_2_cycles = np.arange(4)
delay_sweep_3_cycles = np.arange(4)

delay_sweep_2 = []
for delay in delay_sweep_2_cycles:
    delay_sweep_2.append(soc.cycles2us(delay)*1e3)
delay_sweep_3 = []
for delay in delay_sweep_3_cycles:
    delay_sweep_3.append(soc.cycles2us(delay)*1e3)
print("delay_sweep_2", delay_sweep_2)
print("delay_sweep_3", delay_sweep_3)

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

In [ ]:
play_pulses = protocol_play
for delay_2 in delay_sweep_2:
    for delay_3 in delay_sweep_3:
        print('delays', delay_2, delay_3)
        for init_state in init_states:
            qramExpt = meas.QramProtocol3QTomoExperiment(
                soccfg=soc,
                path=expt_path,
                config_file=config_path,
                ns_address=ns_address,
            )
            print('init_state', init_state, 'play pulses', play_pulses)
            
            span = np.inf
            npts = 1
            if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
            
            qramExpt.cfg.expt = dict(
                start=span,
                step=0,
                expts=1, # number of different time experiments
            
                reps=7500, # number of reps per tomo/time step,
                singleshot_reps=7500, # reps per state for singleshot calibration
                # reps=100, # number of reps per tomo/time step,
                # singleshot_reps=100, #000, # reps per state for singleshot calibration
                post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
                tomo_3q=True, # flag to perform tomo on last time step
                tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
                init_state=init_state,
                play_pulses=play_pulses, # [2, 1, 4, 3]
                cool_qubits=cool_qubits,
                sync_between_swaps=sync_between_swaps,
                sync_between_efs=sync_between_efs,
                
                use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
                use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
                plot_IQ=False,
                
                readout_cool=readout_cool,
                n_init_readout=1,
                n_trig=1,
                init_read_wait_us=10,
                
                full_mux_expt=full_mux_expt,
                resonator_reset=resonator_reset,
            )
            if tomo_qubits == [0, 2, 3]:
                qramExpt.cfg.device.readout.gain[1] = 1e-4
                
            qramExpt.cfg.hw.soc.dacs.delay_chs.delay_ns[2] = delay_2
            qramExpt.cfg.hw.soc.dacs.delay_chs.delay_ns[3] = delay_3

            
            try:
                qramExpt.acquire(progress=True, debug=False)
                success = True
            except Exception:
                print("Pyro traceback:")
                print("".join(Pyro4.util.getPyroTraceback()))
                
            if save_data: saved_files.append(qramExpt.save_data())

In [ ]:
print('init_states =', init_states)
print(saved_files)

In [ ]:
n_tomo_corrected_dict = dict()

all_configs = dict()
i_saved_file = 0
for i_delay_2, delay_2 in enumerate(tqdm(delay_sweep_2)):
    for i_delay_3, delay_3 in enumerate(tqdm(delay_sweep_3)):
        for init_state in init_states:
            saved_file = saved_files[i_saved_file]
            print(saved_file)
            data, attrs = prev_data(saved_file, None, verbose=False)
            pulse_dict = attrs['pulse_dict']
            cfg = AttrDict(attrs['config'])
            assert cfg.expt.init_state == init_state
            all_configs[init_state] = cfg
            play_pulses = cfg.expt.play_pulses
            print('init state Q0 (switch) Q1 (input):', init_state)
            
            n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
            n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
            
            if init_state not in n_tomo_corrected_dict.keys():
                n_tomo_corrected_dict.update({init_state:np.zeros((len(delay_sweep_2), len(delay_sweep_3), 27, 8))})
            
            n_tomo_corrected_dict[init_state][i_delay_2, i_delay_3] = n_tomo_corrected
            i_saved_file += 1

init_states = np.copy(list(n_tomo_corrected_dict.keys()))

In [ ]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})
n_tomo_corrected_dict.update({"delay_sweep_2":delay_sweep_2})
n_tomo_corrected_dict.update({"delay_sweep_3":delay_sweep_3})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

In [ ]:
# RETRIEVE SAVED 2Q TOMO INITIAL STATES FROM MEASUREMENT
# rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202411051014_init_rhoMLE_ZZ_2Q_01.npz'
rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202412051654_init_rhoMLE_ZZ_2Q_01.npz'

print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

rho_MLE_ZZ_dict = dict()
with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
    for key in npzfile.keys():
        rho_MLE_ZZ_dict.update({key:npzfile[key]})
print(f'rho_MLE_ZZ_dict retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

In [429]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

Reopening 3q tomo data

In [ ]:
# RETRIEVING MEASURED n_tomo_corrected
n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_241025\\202412052137_ntomocorrected_3Q_023.npz"
print(f"Using n_tomo_corrected_dict from file {n_tomo_corrected_filepath}")
n_tomo_corrected_dict = dict()

n_tomo_corrected_dict.update({"saved_files":saved_files})
with np.load(n_tomo_corrected_filepath, allow_pickle=True) as npzfile:
    for key in npzfile.keys():
        n_tomo_corrected_dict.update({key:npzfile[key]})

init_states = n_tomo_corrected_dict["init_states"]

print(f'n_tomo_corrected_dict retrieved with init_states\n{init_states}')
all_configs = n_tomo_corrected_dict["all_configs"].item() #[0]
# print(all_configs.keys())


cfg = AttrDict(all_configs[init_states[0]])
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits

print("play_pulses", play_pulses)

Looping MLE

In [519]:
use_init_2q_state = True

In [520]:
rhos_base = []
fidelities_base = []
purities_base = []

rhos_ZZ = []
fidelities_ZZ = []
purities_ZZ = []

In [ ]:
for init_state in init_states:
    print('init state (Q0 (switch) Q1 (input)):', init_state)
    n_tomo_corrected_delays = n_tomo_corrected_dict[init_state]
    print(n_tomo_corrected_delays.shape)
    
    for i_delay_2, delay_2 in enumerate(delay_sweep_2):
        for i_delay_3, delay_3 in enumerate(delay_sweep_3):
            n_tomo_corrected = n_tomo_corrected_delays[i_delay_2, i_delay_3]
            
            if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
            else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
            rho_id = get_qram_qSLR_state_from_rho(init_rho)
            
            # ------------------------------ #
            # Base MLE
            # ------------------------------ #
            
            rho_MLE_base = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=True,
                method='analytical',
                ZZ_correction=False,
            )
         
            fid_base = qt.fidelity(qt.Qobj(rho_MLE_base, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
            purity_base = np.real(np.trace(rho_MLE_base @ rho_MLE_base))
            purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
            
            rhos_base.append(rho_MLE_base)
            fidelities_base.append(fid_base)
            purities_base.append(purity_base)
            
            rho_MLE = rho_MLE_base
        
            # ------------------------------ #
            # MLE with ZZ
            # ------------------------------ #
        
            rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=True,
                method='analytical',
                ZZ_correction=True,
                evol_mats=evol_mats
            )
        
            fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
            purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
            purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
        
            rhos_ZZ.append(rho_MLE_ZZ)
            fidelities_ZZ.append(fid_ZZ)
            purities_ZZ.append(purity_ZZ)

Run loop over virtual Z rotations

In [522]:
use_ZZ_correction = True
show_fids_grid = False

In [523]:
fidelities_opt_rot = []
all_best_phis = []
saveplots = False
# save_append = '_compareMeas'
save_append = ''
phis = [None, None, None]

In [ ]:
phis[0] = np.linspace(0, 360, 19) # Q0
phis[1] = np.linspace(0, 360, 19) # Q2
phis[2] = np.linspace(0, 360, 19) # Q3

print(phis)

In [ ]:
fids_grids_all_delays = np.zeros((len(init_states), len(delay_sweep_2), len(delay_sweep_3), len(phis[0]), len(phis[1]), len(phis[2])))
for i_init_state, init_state in enumerate(init_states):
    print('init state (Q0 (switch) Q1 (input)):', init_state)
    n_tomo_corrected_delays = n_tomo_corrected_dict[init_state]
    
    for i_delay_2, delay_2 in enumerate(delay_sweep_2):
        for i_delay_3, delay_3 in enumerate(delay_sweep_3):
            n_tomo_corrected = n_tomo_corrected_delays[i_delay_2, i_delay_3]
            
            if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
            else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
            rho_id = get_qram_qSLR_state_from_rho(init_rho)
            
            rho_MLE = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=True,
                method='analytical',
                ZZ_correction=use_ZZ_correction,
                evol_mats=None if not use_ZZ_correction else evol_mats,
            )
            
            rho_MLE_rot, best_phis, best_fid, fids_grid = tomo_analysis.opt_virtualZ_MLE(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis, progress=True)
            fids_grids_all_delays[i_init_state, i_delay_2, i_delay_3] = fids_grid
            print("best_phis", best_phis)
            print("best_fid", best_fid)

Go through the fids grids and select the best overall phi

In [527]:
check_init_states = init_states
# check_init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
# check_init_states = ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+i>']

In [ ]:
fids_grids_all_delays.shape

In [ ]:
best_avg_fids_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3)))
best_phi_indices_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3), 3))
best_phis_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3), 3))
best_fids_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3), len(check_init_states)))

# Pick best overall phi for each delay set
for i_delay_2, delay_2 in enumerate(delay_sweep_2):
    for i_delay_3, delay_3 in enumerate(delay_sweep_3):
        best_avg_fid = 0
        best_phis = [0, 0, 0]
        best_phi_indices = [0, 0, 0]
        for iphi0, phi0 in enumerate(phis[0]):
            for iphi1, phi1 in enumerate(phis[1]):
                for iphi2, phi2 in enumerate(phis[2]):
                    avg_fid = 0
                    for i_state, init_state in enumerate(init_states):
                        if init_state not in check_init_states: continue
                        avg_fid += fids_grids_all_delays[i_state, i_delay_2, i_delay_3, iphi0, iphi1, iphi2]
                    avg_fid /= len(check_init_states)
                    if avg_fid > best_avg_fid:
                        best_phis = [phi0, phi1, phi2]
                        best_phi_indices = [iphi0, iphi1, iphi2]
                        best_avg_fid = avg_fid
                        i_check_init_state = 0
                        for i_state, init_state in enumerate(init_states):
                            if init_state not in check_init_states: continue
                            best_fids_all_delays[i_delay_2, i_delay_3, i_check_init_state] = fids_grids_all_delays[i_state, i_delay_2, i_delay_3, iphi0, iphi1, iphi2]
                            i_check_init_state += 1
        best_avg_fids_all_delays[i_delay_2, i_delay_3] = best_avg_fid
        best_phi_indices_all_delays[i_delay_2, i_delay_3] = best_phi_indices
        best_phis_all_delays[i_delay_2, i_delay_3] = best_phis

# Pick best delay set
best_delay_indices = np.unravel_index(np.argmax(best_avg_fids_all_delays), best_avg_fids_all_delays.shape)
best_delays = [delay_sweep_2[best_delay_indices[0]], delay_sweep_3[best_delay_indices[1]]]
best_avg_fid = best_avg_fids_all_delays[best_delay_indices]
best_phi_indices = best_phi_indices_all_delays[best_delay_indices]
best_phis = best_phis_all_delays[best_delay_indices]
best_fids = best_fids_all_delays[best_delay_indices]

print(init_states)
print('best delays', best_delays)
print('best avg fid', best_avg_fid)
print('best fids for check states', best_fids)
print('best phis', best_phis)

In [535]:
outer_sweep = delay_sweep_2
inner_sweep = delay_sweep_3
y_sweep = outer_sweep
x_sweep = inner_sweep

plt.figure()
plt.title(f'Avg Fidelities for Optimal Phis')
plt.xlabel(f'Delays Ch 3 (ns)')
plt.ylabel(f'Delays Ch 2 (ns)')
plt.pcolormesh(x_sweep, y_sweep, best_avg_fids_all_delays, cmap='viridis', shading='auto')
plt.colorbar()
plt.clim(vmin=0.8, vmax=vmax)
plt.show()

In [ ]:
inner_sweep = phis[2]
outer_sweep = phis[1]
y_sweep = outer_sweep
x_sweep = inner_sweep
print(fids_grid_all_delays[i_state, best_delay_indices[0], best_delay_indices[1], i_phi0].shape)
print(f'phi Q0 {phi0}')
plt.figure()
plt.title(f'Fidelities ($\phi_0: ${phi0:0.4})')
plt.xlabel(f'phis Q{tomo_qubits[2]}')
plt.ylabel(f'phis Q{tomo_qubits[1]}')
plt.pcolormesh(x_sweep, y_sweep, fids_grid[i_phi0], cmap='viridis', shading='auto')
plt.colorbar()
plt.clim(vmin=0, vmax=vmax)
plt.show()

## Big qram protocol loop!

In [44]:
cardinal_states = ['|0>', '|1>', '|0+1>', '|0+i>']
init_states = []
for state0 in cardinal_states:
    for state1 in cardinal_states:
        init_states.append(state0 + state1)

# init_states = ['|0+1>|0>', '|0>|0+1>', '|1>|0+1>', '|0+1>|1>', '|0+1>|0+1>']
# init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
# init_states = ['|0+1>|1>', '|1>|1>', '|0>|1>', '|0+1>|0+1>']
# init_states = ['|0+1>|0+1>']

protocol_play = [2, 1, 3, 4] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

sync_between_swaps = True
sync_between_efs = True

tomo_qubits = [0, 2, 3] # which qubit/resonators to do tomo on
save_data = True
saved_files = []
# cool_qubits = [0, 1]
cool_qubits = None

readout_cool = False
full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None

use_IQ_pulse = False # state transfer IQ pulses for the state prep
use_robust_pulses = True # use robust pulses for state prep + tomo

iterations = 5
# iterations = 1

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

init_states (16 states) ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
protocol_play [2, 1, 3, 4]
simultaneous swaps False
simultaneous efs False
readout cool False
full mux expt False
resonator reset None
use robust pulses True
save data True


In [16]:
play_pulses = protocol_play
for it in range(iterations):
    for init_state in init_states:
        qramExpt = meas.QramProtocol3QTomoExperiment(
            soccfg=soc,
            path=expt_path,
            config_file=config_path,
            ns_address=ns_address,
        )
        print('init_state', init_state, 'play pulses', play_pulses)
        
        span = np.inf
        npts = 1
        if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
        
        qramExpt.cfg.expt = dict(
            start=span,
            step=0,
            expts=1, # number of different time experiments
        
            reps=10000, # number of reps per tomo/time step,
            singleshot_reps=10000, # reps per state for singleshot calibration
            # reps=100, # number of reps per tomo/time step,
            # singleshot_reps=100, #000, # reps per state for singleshot calibration
            post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
            tomo_3q=True, # flag to perform tomo on last time step
            tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
            init_state=init_state,
            play_pulses=play_pulses, # [2, 1, 4, 3]
            cool_qubits=cool_qubits,
            sync_between_swaps=sync_between_swaps,
            sync_between_efs=sync_between_efs,
            
            use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
            use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
            plot_IQ=False,
            
            readout_cool=readout_cool,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=10,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )
        if tomo_qubits == [0, 2, 3]:
            qramExpt.cfg.device.readout.gain[1] = 1e-4
        
        try:
            qramExpt.acquire(progress=True, debug=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))
            
        if save_data: saved_files.append(qramExpt.save_data())

init_state |0+1>|1> play pulses [2, 1, 3, 4]
timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 85.38 	 angle (deg): 75.5359964203401 	 threshold ge: 155.46752298940442
Qubit  (2)
ge fidelity (%): 72.7187418741874 	 angle (deg): -100.58348153846732 	 threshold ge: 33.74102261423135
Qubit  (3)
ge fidelity (%): 80.76953995399543 	 angle (deg): -71.39846168479653 	 threshold ge: 50.54743673571738
thresholds=[155.46752298940442, 0, 33.74102261423135, 50.54743673571738],
angles=[75.5359964203401, 0, -100.58348153846732, -71.39846168479653],
ge_avgs=[array([  80.43232851,  -78.90133669,  113.31245224, -206.36965586]), [0, 0, 0, 0], array([-45.46389354,   2.39235562, -54.47715616,  50.6312102 ]), array([-12.67402217,  29.56314502,   3.95206146,  78.96216546])] ,
counts_calib=[[7741, 412, 1209, 69, 478, 25, 59, 7], [1157, 6678, 158, 1123, 93, 653, 24, 114], [1263, 56, 7764, 416, 66, 2, 409, 24], [209, 1009, 1171, 7078, 10, 63, 75, 385], [745, 32, 113, 4, 7477, 420, 1151, 58], [156, 762, 22, 123, 1163, 6608, 169, 997], [164, 3, 808, 45, 1091, 66, 7388, 435], [3

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 82.9993199319932 	 angle (deg): 71.15145457701486 	 threshold ge: 148.52636453694845
Qubit  (2)
ge fidelity (%): 75.87000000000003 	 angle (deg): -97.15522415696806 	 threshold ge: 35.22708537224807
Qubit  (3)
ge fidelity (%): 79.86999999999999 	 angle (deg): -79.8512196808586 	 threshold ge: 53.61420948958115
thresholds=[148.52636453694845, 0, 35.22708537224807, 53.61420948958115],
angles=[71.15145457701486, 0, -97.15522415696806, -79.8512196808586],
ge_avgs=[array([  86.57237923,  -74.46996194,  127.16538842, -193.38079682]), [0, 0, 0, 0], array([-46.70623711,   5.85276473, -53.13768361,  57.08485887]), array([-11.68390811,  29.70107285,  -2.94349668,  78.52864456])] ,
counts_calib=[[7743, 409, 1076, 74, 563, 42, 88, 5], [1224, 6974, 172, 1020, 76, 448, 14, 72], [1051, 46, 7774, 402, 77, 6, 605, 39], [165, 956, 1260, 6966, 11, 62, 93, 487], [842, 46, 114, 6, 7521, 342, 1075, 54], [137, 744, 24, 109, 1236, 6659, 162, 929], [204, 5, 788, 28, 1241, 65, 7301, 

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Saving S:\QRAM\qram_4QR2\data\data_250119\00576_QramProtocol3QTomo.h5


In [17]:
print('init_states =', init_states)
print(saved_files)

init_states = ['|0+1>|1>', '|1>|1>', '|0>|1>', '|0+1>|0+1>']
['S:\\QRAM\\qram_4QR2\\data\\data_250119\\00573_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00574_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00575_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00576_QramProtocol3QTomo.h5']


#### Retrieve saved files and put all the n_tomo_corrected into a more easily accessible file

In [18]:
apply_ps = False
ps_qubits = [0, 1, 2, 3]
ps_adjust = [0]*4

In [19]:
n_tomo_corrected_dict = dict()
all_configs = dict()
for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
    print(saved_file)
    data, attrs = prev_data(saved_file, None)
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    init_state = cfg.expt.init_state
    all_configs[init_state] = cfg
    play_pulses = cfg.expt.play_pulses
    print('init state Q0 (switch) Q1 (input):', init_state)
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
    n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
    
    if init_state not in n_tomo_corrected_dict.keys():
        n_tomo_corrected_dict.update({init_state:[]})
    
    n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

init_states = np.copy(list(n_tomo_corrected_dict.keys()))
for init_state in init_states:
    all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
    n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_11184\3499015544.py: 3

  0%|          | 0/4 [00:00<?, ?it/s]

 C:\Users\slab\AppData\Local\Temp\ipykernel_11184\2316176076.py: 13

S:\QRAM\qram_4QR2\data\data_250119\00573_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00573_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_250119\00574_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00574_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |1>|1>
S:\QRAM\qram_4QR2\data\data_250119\00575_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00575_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|1>
S:\QRAM\qram_4QR2\data\data_250119\00576_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00576_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0+1>|0+1>


In [20]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

Saved n_tomo_corrected_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_250119\202503261440_ntomocorrected_3Q_023.npz


### Round 1a with simultaneous pulses

In [290]:
# saved_files = ['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00515_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00516_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00517_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00518_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00519_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00520_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00521_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00522_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00523_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00524_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00525_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00526_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00527_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00528_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00529_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00530_QramProtocol3QTomo.h5']
# iterations = 4

In [ ]:
# saved_files = []

sync_between_swaps = False
sync_between_efs = False

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

In [ ]:
play_pulses = protocol_play
for it in range(iterations):
    for init_state in init_states:
        qramExpt = meas.QramProtocol3QTomoExperiment(
            soccfg=soc,
            path=expt_path,
            config_file=config_path,
            ns_address=ns_address,
        )
        print('init_state', init_state, 'play pulses', play_pulses)
        
        span = np.inf
        npts = 1
        if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
        
        qramExpt.cfg.expt = dict(
            start=span,
            step=0,
            expts=1, # number of different time experiments
        
            reps=10000, # number of reps per tomo/time step,
            singleshot_reps=10000, # reps per state for singleshot calibration
            # reps=100, # number of reps per tomo/time step,
            # singleshot_reps=100, #000, # reps per state for singleshot calibration
            post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
            tomo_3q=True, # flag to perform tomo on last time step
            tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
            init_state=init_state,
            play_pulses=play_pulses, # [2, 1, 4, 3]
            cool_qubits=cool_qubits,
            sync_between_swaps=sync_between_swaps,
            sync_between_efs=sync_between_efs,
            
            use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
            use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
            plot_IQ=False,
            
            readout_cool=readout_cool,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=10,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )
        if tomo_qubits == [0, 2, 3]:
            qramExpt.cfg.device.readout.gain[1] = 1e-4
        
        try:
            qramExpt.acquire(progress=True, debug=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))
            
        if save_data: saved_files.append(qramExpt.save_data())

In [ ]:
print('init_states =', init_states)
print(saved_files)

#### Retrieve saved files and put all the n_tomo_corrected into a more easily accessible file

In [ ]:
n_tomo_corrected_dict = dict()
all_configs = dict()
for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
    print(saved_file)
    data, attrs = prev_data(saved_file, None)
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    init_state = cfg.expt.init_state
    all_configs[init_state] = cfg
    play_pulses = cfg.expt.play_pulses
    print('init state Q0 (switch) Q1 (input):', init_state)
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
    n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
    
    if init_state not in n_tomo_corrected_dict.keys():
        n_tomo_corrected_dict.update({init_state:[]})
    
    n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

init_states = np.copy(list(n_tomo_corrected_dict.keys()))
for init_state in init_states:
    all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
    n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

In [ ]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

### Iterate over many rounds of swap applications

In [47]:
init_states = []
for state0 in cardinal_states:
    for state1 in cardinal_states:
        init_states.append(state0 + state1)

sync_between_swaps = True
sync_between_efs = True

n_sweep = [0, 1, 2, 3, 4, 5]
# n_sweep = [4, 5, 6, 7]
iterations = 5
# iterations = 2

saved_files_nsweep = []
n_tomo_corrected_filepaths = [] # 1 for each n_forward_backward value

print("n_sweep", n_sweep)
print("iterations", iterations)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')


n_sweep [0, 1, 2, 3, 4, 5]
iterations 5
simultaneous swaps False
simultaneous efs False
readout cool False
full mux expt False
resonator reset None
use robust pulses True
save data True


In [48]:
# n_forward_backward = 1
for i_n, n_forward_backward in enumerate(n_sweep):
    print("############################################")
    print("STARTING n_forward_backward", n_forward_backward)
    saved_files_nsweep.append([])
    saved_files = saved_files_nsweep[-1]
    protocol_play = []
    for n in range(n_forward_backward):
        protocol_play.append([2, 1, 1, 2])
    protocol_play.append([2, 1, 3, 4])
    protocol_play = np.array(protocol_play).flatten().tolist()

    print(f'init_states ({len(init_states)} states)', init_states)
    print('protocol_play', protocol_play)

    play_pulses = protocol_play
    for it in range(iterations):
        print("-------------------")
        print("Starting iteration", it, "for n_forward_backward", n_forward_backward)
        for init_state in tqdm(init_states):
            qramExpt = meas.QramProtocol3QTomoExperiment(
                soccfg=soc,
                path=expt_path,
                config_file=config_path,
                ns_address=ns_address,
            )
            print('init_state', init_state, 'play pulses', play_pulses)

            span = np.inf
            npts = 1
            if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))

            qramExpt.cfg.expt = dict(
                start=span,
                step=0,
                expts=1, # number of different time experiments

                reps=10000, # number of reps per tomo/time step,
                singleshot_reps=10000, # reps per state for singleshot calibration
                # reps=100, # number of reps per tomo/time step,
                # singleshot_reps=100, #000, # reps per state for singleshot calibration
                post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
                tomo_3q=True, # flag to perform tomo on last time step
                tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
                init_state=init_state,
                play_pulses=play_pulses, # [2, 1, 4, 3]
                cool_qubits=cool_qubits,
                sync_between_swaps=sync_between_swaps,
                sync_between_efs=sync_between_efs,

                use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
                use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
                plot_IQ=False,

                readout_cool=readout_cool,
                n_init_readout=1,
                n_trig=1,
                init_read_wait_us=10,

                full_mux_expt=full_mux_expt,
                resonator_reset=resonator_reset,
            )
            if tomo_qubits == [0, 2, 3]:
                qramExpt.cfg.device.readout.gain[1] = 1e-4

            try:
                qramExpt.acquire(progress=True, debug=False)
                success = True
            except Exception:
                print("Pyro traceback:")
                print("".join(Pyro4.util.getPyroTraceback()))

            if save_data: saved_files.append(qramExpt.save_data())
    print("For n_forward_backward", n_forward_backward, "saved_files", saved_files)
    print()


    print("............")
    print("Assembling n_tomo_corrected_dict")
    n_tomo_corrected_dict = dict()
    all_configs = dict()
    for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
        print(saved_file)
        data, attrs = prev_data(saved_file, None)
        pulse_dict = attrs['pulse_dict']
        cfg = AttrDict(attrs['config'])
        init_state = cfg.expt.init_state
        all_configs[init_state] = cfg
        play_pulses = cfg.expt.play_pulses
        print('init state Q0 (switch) Q1 (input):', init_state)

        n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
        n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)

        if init_state not in n_tomo_corrected_dict.keys():
            n_tomo_corrected_dict.update({init_state:[]})

        n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

    init_states = np.copy(list(n_tomo_corrected_dict.keys()))
    for init_state in init_states:
        all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
        n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

    # SAVING MEASURED n_tomo_corrected
    n_tomo_corrected_dict.update({"saved_files":saved_files})
    n_tomo_corrected_dict.update({"init_states":list(init_states)})
    n_tomo_corrected_dict.update({"all_configs":all_configs})

    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d%H%M")
    qA, qB, qC = tomo_qubits
    n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
    np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
    print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')
    n_tomo_corrected_filepaths.append(n_tomo_corrected_filepath)

############################################
STARTING n_forward_backward 0
init_states (16 states) ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
protocol_play [2, 1, 3, 4]
-------------------
Starting iteration 0 for n_forward_backward 0


Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_11184\1008061538.py: 20

  0%|          | 0/16 [00:00<?, ?it/s]

init_state |0>|0> play pulses [2, 1, 3, 4]
timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Saving S:\QRAM\qram_4QR2\data\data_250119\01056_QramProtocol3QTomo.h5
For n_forward_backward 5 saved_files ['S:\\QRAM\\qram_4QR2\\data\\data_250119\\00977_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00978_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00979_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00980_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00981_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00982_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00983_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00984_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00985_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00986_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00987_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00988_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\

  0%|          | 0/80 [00:00<?, ?it/s]

S:\QRAM\qram_4QR2\data\data_250119\00977_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00977_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0>
S:\QRAM\qram_4QR2\data\data_250119\00978_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00978_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|1>
S:\QRAM\qram_4QR2\data\data_250119\00979_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00979_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_250119\00980_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_250119\00980_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+i>
S:\QRAM\

In [49]:
print('init_states =', init_states)
for i_n, n_forward_backward in enumerate(n_sweep):
    print("n_forward_backward", n_forward_backward)
    print("saved_files", saved_files_nsweep[i_n])
    print("n_tomo_corrected_filepath", n_tomo_corrected_filepaths[i_n])

init_states = ['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
n_forward_backward 0
saved_files ['S:\\QRAM\\qram_4QR2\\data\\data_250119\\00577_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00578_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00579_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00580_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00581_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00582_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00583_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00584_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00585_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00586_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_250119\\00587_QramProtocol3QT

### Retrieve saved files and put all the n_tomo_corrected into a more easily accessible file

In [ ]:
n_tomo_corrected_dict = dict()
all_configs = dict()
for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
    print(saved_file)
    data, attrs = prev_data(saved_file, None)
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    init_state = cfg.expt.init_state
    all_configs[init_state] = cfg
    play_pulses = cfg.expt.play_pulses
    print('init state Q0 (switch) Q1 (input):', init_state)
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
    n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
    
    if init_state not in n_tomo_corrected_dict.keys():
        n_tomo_corrected_dict.update({init_state:[]})
    
    n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

init_states = np.copy(list(n_tomo_corrected_dict.keys()))
for init_state in init_states:
    all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
    n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

In [ ]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

### Loop analysis

In [22]:
# saved_files = ['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00191_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00192_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00193_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00194_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00195_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00196_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00197_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00198_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00199_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00200_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00201_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00202_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00203_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00204_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00205_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00206_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00207_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00208_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00209_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00210_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00211_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00212_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00213_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00214_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00215_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00216_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00217_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00218_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00219_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00220_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00221_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00222_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00223_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00224_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00225_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00226_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00227_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00228_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00229_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00230_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00231_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00232_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00233_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00234_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00235_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00236_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00237_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00238_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00239_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00240_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00241_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00242_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00243_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00244_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00245_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00246_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00247_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00248_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00249_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00250_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00251_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00252_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00253_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00254_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00255_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00256_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00257_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00258_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00259_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00260_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00261_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00262_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00263_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00264_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00265_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00266_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00267_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00268_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00269_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00270_QramProtocol3QTomo.h5']

In [21]:
# RETRIEVE SAVED 2Q TOMO INITIAL STATES FROM MEASUREMENT
# rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202411051014_init_rhoMLE_ZZ_2Q_01.npz'
rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_250119\\202502111059_init_rhoMLE_ZZ_2Q_01.npz'

print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

rho_MLE_ZZ_dict = dict()
with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
    for key in npzfile.keys():
        rho_MLE_ZZ_dict.update({key:npzfile[key]})
print(f'rho_MLE_ZZ_dict retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

Using rho_MLE_ZZ for Q0/Q1 from file S:\QRAM\qram_4QR2\data\data_250119\202502111059_init_rhoMLE_ZZ_2Q_01.npz
rho_MLE_ZZ_dict retrieved with init_states dict_keys(['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>'])


In [22]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

#### Reopening 3q tomo data

In [ ]:
# RETRIEVING MEASURED n_tomo_corrected
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202503270412_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202503270800_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202503271149_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202503271538_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202503271928_ntomocorrected_3Q_023.npz"
n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202503272318_ntomocorrected_3Q_023.npz"


# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202502120248_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202502120636_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202502121025_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202502121415_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202502130312_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202502130703_ntomocorrected_3Q_023.npz"


print(f"Using n_tomo_corrected_dict from file {n_tomo_corrected_filepath}")
n_tomo_corrected_dict = dict()

with np.load(n_tomo_corrected_filepath, allow_pickle=True) as npzfile:
    for key in npzfile.keys():
        n_tomo_corrected_dict.update({key:npzfile[key]})

# print(n_tomo_corrected_dict.keys())
# print(n_tomo_corrected_dict["saved_files"])
init_states = n_tomo_corrected_dict["init_states"]

if 'saved_files' in init_states: init_states = init_states[:-1]

print(f'n_tomo_corrected_dict retrieved with init_states\n{init_states}')
all_configs = n_tomo_corrected_dict["all_configs"].item() #[0]


cfg = AttrDict(all_configs[init_states[0]])
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits


print("play_pulses", play_pulses)

Using n_tomo_corrected_dict from file S:\QRAM\qram_4QR2\data\data_250119\202503261440_ntomocorrected_3Q_023.npz
n_tomo_corrected_dict retrieved with init_states
['|0+1>|1>' '|1>|1>' '|0>|1>' '|0+1>|0+1>']
play_pulses [2, 1, 3, 4]


#### Reload evol mats

In [24]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

tomo_qubits = [0, 2, 3]

# evol_mats_path = os.path.join(os.getcwd(), 'evol_mats')
evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"
qA, qB, qC = tomo_qubits
evol_mats_filename = f'evol_mats_{qA}{qB}{qC}.npz'
evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
# evol_mats_file_path = os.path.join(evol_mats_path, f'evol_mats_{qA}{qB}{qC}_test.npz')
print(f'Using evol mats from file {evol_mats_file_path}')

evol_mats = dict()
with np.load(evol_mats_file_path) as npzfile:
    for key in npzfile.keys():
        evol_mats.update({key:npzfile[key]})
# print(evol_mats)

Using evol mats from file S:\QRAM\qram_4QR2\evol_mats\evol_mats_023.npz


#### MLE timing test!

In [30]:
def test_base(n_tomo):
    rho_MLE = tomo_analysis.run_MLE_analytical(
        n_tomo=n_tomo,
        ZZ_correction=False,
        evol_mats=None,
    )
    # print("hello")
    return rho_MLE

def test_ZZ(n_tomo):
    rho_MLE = tomo_analysis.run_MLE_analytical(
        n_tomo=n_tomo,
        ZZ_correction=True,
        evol_mats=evol_mats,
    )
    return rho_MLE


# from timeit import Timer
import timeit
loop = 50

timing_base = []
timing_ZZ = []

for init_state in init_states:
    print('init state (Q0 (switch) Q1 (input)):', init_state)
    n_tomo = n_tomo_corrected_dict[init_state]
    elapsed = timeit.timeit("test_base(n_tomo)", globals=globals(), number=loop)
    print("\telapsed time base", elapsed, f"({elapsed/loop} s per call)")
    timing_base.append(elapsed)

    elapsed = timeit.timeit("test_ZZ(n_tomo)", globals=globals(), number=loop)
    print("\telapsed time base", elapsed, f"({elapsed/loop} s per call)")
    timing_ZZ.append(elapsed)

print("total time base", timing_base)
print("average time base per call", np.average(timing_base) / loop)

print("total time ZZ", timing_ZZ)
print("average time ZZ per call", np.average(timing_ZZ) / loop)

init state (Q0 (switch) Q1 (input)): |0>|0>
	elapsed time base 6.753186000016285 (0.1350637200003257 s per call)


 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1178ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1183

	elapsed time base 13.786297100014053 (0.27572594200028105 s per call)
init state (Q0 (switch) Q1 (input)): |0>|1>
	elapsed time base 6.286864600027911 (0.12573729200055822 s per call)
	elapsed time base 13.541306600003736 (0.2708261320000747 s per call)
init state (Q0 (switch) Q1 (input)): |0>|0+1>
	elapsed time base 6.334593000006862 (0.12669186000013724 s per call)
	elapsed time base 13.524261699989438 (0.27048523399978874 s per call)
init state (Q0 (switch) Q1 (input)): |0>|0+i>
	elapsed time base 6.494665200007148 (0.12989330400014296 s per call)
	elapsed time base 13.157072400004836 (0.2631414480000967 s per call)
init state (Q0 (switch) Q1 (input)): |1>|0>
	elapsed time base 6.518410700024106 (0.13036821400048212 s per call)
	elapsed time base 13.181883999990532 (0.26363767999981064 s per call)
init state (Q0 (switch) Q1 (input)): |1>|1>
	elapsed time base 6.488733000005595 (0.1297746600001119 s per call)
	elapsed time base 13.318814300000668 (0.26637628600001334 s per call)
ini

#### Looping MLE

In [25]:
rhos_base = []
rhos_ZZ = []

In [26]:
for init_state in init_states:
    print('init state (Q0 (switch) Q1 (input)):', init_state)

    n_tomo_corrected = n_tomo_corrected_dict[init_state]
    
    # ------------------------------ #
    # Base MLE
    # ------------------------------ #
    
    rho_MLE_base = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_corrected,
        n_calib=None,
        correct_readout=False,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=False,
    )
    rhos_base.append(rho_MLE_base)

    # ------------------------------ #
    # MLE with ZZ
    # ------------------------------ #

    rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_corrected,
        n_calib=None,
        correct_readout=False,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=True,
        evol_mats=evol_mats
    )
    rhos_ZZ.append(rho_MLE_ZZ)

init state (Q0 (switch) Q1 (input)): |0+1>|1>


 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1178ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1183

init state (Q0 (switch) Q1 (input)): |1>|1>
init state (Q0 (switch) Q1 (input)): |0>|1>
init state (Q0 (switch) Q1 (input)): |0+1>|0+1>


In [27]:
print('init_states=', init_states)
print(len(init_states))

init_states= ['|0+1>|1>' '|1>|1>' '|0>|1>' '|0+1>|0+1>']
4


Define function to get the best phi from a fid grid

In [28]:
def get_best_phis(fids_grids, phis, init_states, check_init_states=None, plot_fid_grids=False, verbose=True):
    if check_init_states is None: check_init_states = init_states
    fids_grids_reshaped = []
    for i_state in range(len(init_states)):
        grid_shape = fids_grids[i_state].shape
        # print(grid_shape)
        fids_grid = np.copy(fids_grids[i_state]).tolist()
        # print(init_states[i_state], grid_shape)
        if grid_shape[0] == 1:
            for i in range(grid_shape[1]-1):
                fids_grid.append(fids_grid[0])
        # print(np.array(fids_grid).shape)
        fids_grids_reshaped.append(fids_grid)

    fids_grids_reshaped = np.array(fids_grids_reshaped)
    print(fids_grids_reshaped.shape)

    best_avg_fid = 0
    best_fids = []
    best_phis = [0, 0, 0]
    best_phi_indices = [0, 0, 0]
    individual_best_fids = np.zeros((len(init_states)))
    individual_best_phis = np.zeros((len(init_states), 3))
    for iphi0, phi0 in enumerate(phis[0]):
        for iphi1, phi1 in enumerate(phis[1]):
            for iphi2, phi2 in enumerate(phis[2]):
                avg_fid = 0
                for i_state, init_state in enumerate(init_states):
                    if init_state not in check_init_states: continue
                    fid = fids_grids_reshaped[i_state][iphi0, iphi1, iphi2]
                    avg_fid += fid
                    if fid > individual_best_fids[i_state]:
                        individual_best_fids[i_state] = fid
                        individual_best_phis[i_state] = [phi0, phi1, phi2]
                avg_fid /= len(check_init_states)
                if avg_fid > best_avg_fid:
                    best_phis = [phi0, phi1, phi2]
                    best_phi_indices = [iphi0, iphi1, iphi2]
                    best_avg_fid = avg_fid

    inner_sweep = phis[2]
    outer_sweep = phis[1]
    y_sweep = outer_sweep
    x_sweep = inner_sweep
    
    if plot_fid_grids:
        vmax = 1.0
        for i_state in range(len(init_states)):
            # print(init_states[i_state])
            # print(f'best phi Q0 {best_phis[0]}')
            plt.figure()
            # plt.title(f'Fidelities {init_states[i_state]} ($\phi_0: ${best_phis[0]:0.4})')
            plt.xlabel(f'phis Q{tomo_qubits[2]}')
            plt.ylabel(f'phis Q{tomo_qubits[1]}')
            plt.pcolormesh(x_sweep, y_sweep, fids_grids_reshaped[i_state, best_phi_indices[0], :, :], cmap='viridis', shading='auto')
            plt.colorbar()
            plt.clim(vmin=0, vmax=vmax)
            plt.show()


    for i_state, init_state in enumerate(init_states):
        if init_state not in check_init_states: continue
        best_fids.append(fids_grids_reshaped[i_state][best_phi_indices[0], best_phi_indices[1], best_phi_indices[2]])

    if verbose:
        print('check states', check_init_states)
        print('Best fids with individual rotations', individual_best_fids.tolist())
        print('Avg of best fids with individual rotations', np.average(individual_best_fids))
        print('Phis for best fids with individual rotations', individual_best_phis.tolist())
        print('Best avg fid with overall optimal phis', best_avg_fid)
        print('Fids for check states using overall optimal phis', best_fids)
        print('Overall optimal phis', best_phis)

    return best_phis


#### Run loop over virtual Z rotations

In [29]:
show_fids_grid = False

In [30]:
saveplots = False
# save_append = '_compareMeas'
save_append = ''

In [31]:
# fids_grids_dict structure: fids_grids_dict[name=base/ZZ_vs_meas/ideal][coarse/med/fine][fids_grids/phis]
fids_grids_dict = dict(
    base_vs_meas=dict(),
    ZZ_vs_meas=dict(),
    base_vs_ideal=dict(),
    ZZ_vs_ideal=dict(),
)

for name in fids_grids_dict.keys():
    fids_grids_dict[name].update(dict(
        coarse=dict(fids_grids=[], phis=[]),
        med=dict(fids_grids=[], phis=[]),
        fine=dict(fids_grids=[], phis=[])
        ))

In [33]:
# coarse scan (24 deg), med scan (3 deg), fine scan (0.5 deg)

# COARSE SCAN
span_dict = dict(
    coarse=360,
    med=30,
    fine=5
)

npts_dict = dict(
    coarse=15+1,
    med=10+1,
    fine=10+1,
)

In [34]:
phis = None
prev_opt_phis = None
# for use_init_2q_state in [False]:
#         for use_ZZ_correction in [False]:
for use_init_2q_state in [True, False]:
    for use_ZZ_correction in [True, False]:
        print(f"STARTING use_init_2q_state: {use_init_2q_state}, use_ZZ_correction: {use_ZZ_correction}")
        for scan in ['coarse', 'med', 'fine']:
            if scan == 'coarse':
                start_phis = [0]*3
                end_phis = [span_dict[scan]]*3
            else:
                start_phis = [prev_opt_phi - span_dict[scan]/2 for prev_opt_phi in prev_opt_phis]
                end_phis = [prev_opt_phi + span_dict[scan]/2 for prev_opt_phi in prev_opt_phis] 
            phis = [
                np.linspace(start_phis[0], end_phis[0], int(npts_dict[scan])), # Q0
                np.linspace(start_phis[1], end_phis[1], int(npts_dict[scan])), # Q2
                np.linspace(start_phis[2], end_phis[2], int(npts_dict[scan])), # Q3
            ]
            print("Starting scan", scan, "with phis", phis)
            fids_grids = np.zeros((len(init_states), len(phis[0]), len(phis[1]), len(phis[2])))
            fidelities_opt_rot = []
            all_best_phis = []
            for i_init_state, init_state in enumerate(tqdm(init_states)):
                # print('init state (Q0 (switch) Q1 (input)):', init_state)
                
                if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
                else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
                rho_id = get_qram_qSLR_state_from_rho(init_rho)
                # ------------------------------ #
                # MLE with opt rot
                # ------------------------------ #
                if use_ZZ_correction:
                    rho_MLE = rhos_ZZ[i_init_state]
                else:
                    rho_MLE = rhos_base[i_init_state]
                # print(rho_MLE)
    
                # if init_state == '|0+1>|0+1>' or init_state == '|0+1>|1>': phis[0] = np.linspace(0, 360, 30)
    
                rho_MLE_rot, best_phis, best_fid, fids_grid = tomo_analysis.opt_virtualZ_MLE(
                    qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis, progress=False, verbose=False)
                fids_grids[i_init_state] = fids_grid
    
                inner_sweep = phis[2]
                outer_sweep = phis[1]
                y_sweep = outer_sweep
                x_sweep = inner_sweep
    
                # vmax = np.max(fids_grid)
                vmax = 1.0
                # for i_phi0, phi0 in enumerate(phis[0]):
                if show_fids_grid:
                    for i_phi0, phi0 in enumerate([phis[0][0]]):
                        print(fids_grid[i_phi0].shape)
                        print(f'phi Q0 {phi0}')
                        plt.figure()
                        plt.title(f'Fidelities ($\phi_0: ${phi0:0.4})')
                        plt.xlabel(f'phis Q{tomo_qubits[2]}')
                        plt.ylabel(f'phis Q{tomo_qubits[1]}')
                        plt.pcolormesh(x_sweep, y_sweep, fids_grid[i_phi0], cmap='viridis', shading='auto')
                        plt.colorbar()
                        plt.clim(vmin=0, vmax=vmax)
                        plt.show()
            
                z_phi123 = tomo_analysis.z_gate_nq(best_phis)
                rho_MLE_rot = (z_phi123*qt.Qobj(rho_MLE, dims=rho_id.dims)*z_phi123.dag()).unit()
                fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2
                
                purity_MLE = np.real(np.trace(rho_MLE @ rho_MLE))
                purity_MLE_rot = np.real(np.trace(rho_MLE_rot.full() @ rho_MLE_rot.full()))
                purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
                # print(f'Fidelity (after rotation): {fid_rot}')
                # print(f'Purity (after rotation): {purity_MLE_rot}')
                # print(f'Target purity: {purity_id}')
                # print(f'Best rotation: {best_phis}')
                
                # if saveplots: savetitle=saved_file[:-3]+f'_ZZcorrect_optrot{save_append}.svg'
                # else: savetitle = None
                # show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE with ZZ Correction, Phase Optimized (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]}), Init {init_state}, Play {play_pulses}', savetitle=savetitle, cmax=0.5)
                
                fidelities_opt_rot.append(best_fid)
                all_best_phis.append(best_phis)
                
                # print('\n\n\n')
            if use_ZZ_correction and use_init_2q_state:
                name = "ZZ_vs_meas"
            elif use_ZZ_correction and not use_init_2q_state:
                name = "ZZ_vs_ideal"
            elif not use_ZZ_correction and use_init_2q_state:
                name = "base_vs_meas"
            elif not use_ZZ_correction and not use_init_2q_state:
                name = "base_vs_ideal"

            fids_grids_dict[name][scan]["fids_grids"] = fids_grids
            fids_grids_dict[name][scan]["phis"] = phis

            prev_opt_phis = get_best_phis(fids_grids, phis, init_states, check_init_states=None, plot_fid_grids=False, verbose=True)
            print()

STARTING use_init_2q_state: True, use_ZZ_correction: True
Starting scan coarse with phis [array([  0.,  24.,  48.,  72.,  96., 120., 144., 168., 192., 216., 240.,
       264., 288., 312., 336., 360.]), array([  0.,  24.,  48.,  72.,  96., 120., 144., 168., 192., 216., 240.,
       264., 288., 312., 336., 360.]), array([  0.,  24.,  48.,  72.,  96., 120., 144., 168., 192., 216., 240.,
       264., 288., 312., 336., 360.])]


Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_11184\4112775247.py: 24

  0%|          | 0/4 [00:00<?, ?it/s]

(4, 16, 16, 16)
check states ['|0+1>|1>' '|1>|1>' '|0>|1>' '|0+1>|0+1>']
Best fids with individual rotations [0.9806323295706587, 0.9684450879614129, 0.9517316013481973, 0.9793091449341286]
Avg of best fids with individual rotations 0.9700295409535993
Phis for best fids with individual rotations [[144.0, 48.0, 192.0], [336.0, 264.0, 240.0], [144.0, 48.0, 96.0], [0.0, 168.0, 96.0]]
Best avg fid with overall optimal phis 0.9424587873145535
Fids for check states using overall optimal phis [0.9489164208336082, 0.9154229957519736, 0.9430834471647233, 0.962412285507909]
Overall optimal phis [336.0, 168.0, 120.0]

Starting scan med with phis [array([321., 324., 327., 330., 333., 336., 339., 342., 345., 348., 351.]), array([153., 156., 159., 162., 165., 168., 171., 174., 177., 180., 183.]), array([105., 108., 111., 114., 117., 120., 123., 126., 129., 132., 135.])]


  0%|          | 0/4 [00:00<?, ?it/s]

...

(4, 16, 16, 16)
check states ['|0+1>|1>' '|1>|1>' '|0>|1>' '|0+1>|0+1>']
Best fids with individual rotations [0.9315987415128701, 0.9306041528537238, 0.9478691873252558, 0.9642786666124566]
Avg of best fids with individual rotations 0.9435876870760765
Phis for best fids with individual rotations [[192.0, 120.0, 216.0], [168.0, 216.0, 192.0], [120.0, 360.0, 72.0], [0.0, 168.0, 72.0]]
Best avg fid with overall optimal phis 0.9420613328541284
Fids for check states using overall optimal phis [0.9255029268893649, 0.9306010743758767, 0.9478630293717885, 0.9642783007794836]
Overall optimal phis [360.0, 168.0, 72.0]

Starting scan med with phis [array([345., 348., 351., 354., 357., 360., 363., 366., 369., 372., 375.]), array([153., 156., 159., 162., 165., 168., 171., 174., 177., 180., 183.]), array([57., 60., 63., 66., 69., 72., 75., 78., 81., 84., 87.])]


  0%|          | 0/4 [00:00<?, ?it/s]

(4, 11, 11, 11)
check states ['|0+1>|1>' '|1>|1>' '|0>|1>' '|0+1>|0+1>']
Best fids with individual rotations [0.9385951051190763, 0.9306033198665252, 0.9478684041457595, 0.9680569722135872]
Avg of best fids with individual rotations 0.946280950336237
Phis for best fids with individual rotations [[372.0, 174.0, 81.0], [351.0, 183.0, 60.0], [363.0, 180.0, 78.0], [366.0, 168.0, 60.0]]
Best avg fid with overall optimal phis 0.9439606551824355
Fids for check states using overall optimal phis [0.9344342085616855, 0.9305998415057452, 0.9478630166299524, 0.9629455540323592]
Overall optimal phis [366.0, 162.0, 66.0]

Starting scan fine with phis [array([363.5, 364. , 364.5, 365. , 365.5, 366. , 366.5, 367. , 367.5,
       368. , 368.5]), array([159.5, 160. , 160.5, 161. , 161.5, 162. , 162.5, 163. , 163.5,
       164. , 164.5]), array([63.5, 64. , 64.5, 65. , 65.5, 66. , 66.5, 67. , 67.5, 68. , 68.5])]


  0%|          | 0/4 [00:00<?, ?it/s]

(4, 11, 11, 11)
check states ['|0+1>|1>' '|1>|1>' '|0>|1>' '|0+1>|0+1>']
Best fids with individual rotations [0.9386757914262212, 0.930603922021503, 0.947868269949023, 0.9672427094649146]
Avg of best fids with individual rotations 0.9460976732154154
Phis for best fids with individual rotations [[368.5, 159.5, 68.5], [366.0, 163.0, 67.5], [368.5, 162.5, 68.0], [363.5, 164.5, 63.5]]
Best avg fid with overall optimal phis 0.9440158983249691
Fids for check states using overall optimal phis [0.9354465748041493, 0.9305984776567089, 0.9478630140385487, 0.9621555268004698]
Overall optimal phis [365.5, 162.5, 68.0]



#### Save fids grids

In [35]:
fids_grids_dict.update(n_tomo_corrected_dict)
if "allow_pickle" in fids_grids_dict: fids_grids_dict.pop("allow_pickle")

In [107]:
# SAVING FIDS GRIDS
# n_tomo_corrected_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202412131029_ntomocorrected_3Q_023.npz'
fids_grids_filepath = n_tomo_corrected_filepath[:-4] + '_fids_grids.npz'
np.savez(fids_grids_filepath, allow_pickle=True, **fids_grids_dict)
print(f'Saved fids_grids_dict for Q{qA} Q{qB} Q{qC} to file {fids_grids_filepath} with keys\n{fids_grids_dict.keys()}')

Saved fids_grids_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_250119\202502130703_ntomocorrected_3Q_023_fids_grids.npz with keys
dict_keys(['base_vs_meas', 'ZZ_vs_meas', 'base_vs_ideal', 'ZZ_vs_ideal', '|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>', 'saved_files', 'init_states', 'all_configs'])


Reopening fids grids and 3q tomo data

In [36]:
# RETRIEVING MEASURED n_tomo_corrected and fids grids
fids_grids_filepath = "S:\QRAM\qram_4QR2\data\data_250119\\202502120248_ntomocorrected_3Q_023_fids_grids.npz"

print(f"Using fids_grids_dict from file {fids_grids_filepath}")
fids_grids_dict = dict()

with np.load(fids_grids_filepath, allow_pickle=True) as npzfile:
    for key in npzfile.keys():
        fids_grids_dict.update({key:npzfile[key]})

for key in fids_grids_dict.keys():
    if fids_grids_dict[key].dtype == 'O':
        fids_grids_dict[key] = fids_grids_dict[key].item()

init_states = fids_grids_dict["init_states"]

if 'saved_files' in init_states: init_states = init_states[:-1]

print(f'fids_grids_dict retrieved with init_states\n{init_states}')
all_configs = fids_grids_dict["all_configs"]
print(all_configs.keys())

cfg = AttrDict(all_configs[init_states[0]])
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits

print("play_pulses", play_pulses)

Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_250119\202502120248_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
dict_keys(['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>'])
play_pulses [2, 1, 3, 4]


Select the fids grids you want to use to display/calculate best average fidelities

Go through the fids grids and select the best overall phi

In [36]:
use_init_2q_state = True
use_ZZ_correction = True

In [37]:
scan = 'fine'
# scan = 'coarse'

# fids_grids_dict structure: fids_grids_dict[name=base/ZZ_vs_meas/ideal][coarse/med/fine][fids_grids/phis]
if use_ZZ_correction and use_init_2q_state:
    name = "ZZ_vs_meas"
elif use_ZZ_correction and not use_init_2q_state:
    name = "ZZ_vs_ideal"
elif not use_ZZ_correction and use_init_2q_state:
    name = "base_vs_meas"
elif not use_ZZ_correction and not use_init_2q_state:
    name = "base_vs_ideal"

fids_grids = fids_grids_dict[name][scan]["fids_grids"]
phis = fids_grids_dict[name][scan]["phis"]

best_phis = get_best_phis(fids_grids, phis, init_states, check_init_states=None, plot_fid_grids=True)

(4, 11, 11, 11)
check states ['|0+1>|1>' '|1>|1>' '|0>|1>' '|0+1>|0+1>']
Best fids with individual rotations [0.9487056195112673, 0.9125563193381487, 0.9430614670569966, 0.9798853716549706]
Avg of best fids with individual rotations 0.9460521943903458
Phis for best fids with individual rotations [[339.5, 168.5, 110.5], [339.5, 170.5, 110.5], [344.5, 165.5, 110.5], [344.5, 165.5, 105.5]]
Best avg fid with overall optimal phis 0.9439732442095387
Fids for check states using overall optimal phis [0.9475971716335759, 0.9091160589934996, 0.9427744866986546, 0.9764052595124246]
Overall optimal phis [342.5, 167.5, 108.5]


In [38]:
print(np.array(fids_grids).shape)
print(len(init_states))

(4, 11, 11, 11)
4


Plot all the tomos for the best phis

In [713]:
# best_phis = [360.0, 360.0, 235.86206896551724]
# best_phis = [0]*3

In [39]:
saveplots = False

In [40]:
z_phi123 = tomo_analysis.z_gate_nq(best_phis)
best_fids = []

print()
for i_state, init_state in enumerate(init_states):
    # if init_state not in check_init_states: continue
    print('init_state', init_state)
    if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
    else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
    rho_id = get_qram_qSLR_state_from_rho(init_rho)
    
    if use_ZZ_correction: rho_MLE_rot = (z_phi123*qt.Qobj(rhos_ZZ[i_state], dims=rho_id.dims)*z_phi123.dag()).unit()
    else: rho_MLE_rot = (z_phi123*qt.Qobj(rhos_base[i_state], dims=rho_id.dims)*z_phi123.dag()).unit()
    fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2
    best_fids.append(fid_rot)

    print(rho_MLE_rot, rho_id)
    
    purity_MLE_rot = np.real(np.trace(rho_MLE_rot.full() @ rho_MLE_rot.full()))
    purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
    print(f'Fidelity (after rotation): {fid_rot}')
    print(f'Purity (after rotation): {purity_MLE_rot}')
    print(f'Target purity: {purity_id}')
    
    if saveplots: savetitle=saved_file[:-3]+f'{"_ZZcorrect" if use_ZZ_correction else ""}_optrot{save_append}.svg'
    else: savetitle = None
    
    init_state_title = ''
    for c in init_state:
        if c != '>': init_state_title += c
        else: init_state_title += '\\rangle'
    tomo_analysis.show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE {"with ZZ" if use_ZZ_correction else "Base"}, Phase Corrected (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})\nInit ${init_state_title}$, Play {play_pulses}', savetitle=savetitle, cmax=0.5)
    print()
print('init_states =', init_states)
print('best fids')
print(best_fids)
print('average best fid', np.average(best_fids))
print('used best_phis', best_phis)


init_state |0+1>|1>
Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 9.58313829e-03+0.00000000e+00j  4.65300124e-03-1.35267915e-03j
   2.20723277e-02-1.79698671e-02j  2.69337075e-03-2.27103479e-03j
  -3.16699932e-03+7.84100269e-03j  6.45062452e-03-1.10723672e-03j
  -2.12179370e-03-6.25392826e-04j -1.09667214e-03+7.95216949e-04j]
 [ 4.65300124e-03+1.35267915e-03j  2.75654496e-03+0.00000000e+00j
   1.07630555e-02+4.94969487e-03j  1.56242734e-03-9.02114772e-04j
  -2.07897043e-03+4.82292927e-03j  2.68991315e-03+1.26969377e-02j
  -9.26446339e-04-5.94507770e-04j -5.67946807e-04+2.38509876e-04j]
 [ 2.20723277e-02+1.79698671e-02j  1.07630555e-02-4.94969487e-03j
   4.68682915e-01+0.00000000e+00j  4.80753131e-03+3.54989615e-03j
   2.38199066e-02-1.92584578e-02j  4.46523704e-01-7.00041894e-02j
  -3.54244907e-03-6.02336662e-03j -4.39309280e-03-2.92938242e-03j]
 [ 2.69337075e-03+2.27103479e-03j  1.56242734e-03+9.02114772e-04j
   4.80753131e-0

## Run analysis

### Reopen old data

In [20]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

In [ ]:
saved_files = []
filename = 'S:\QRAM\qram_4QR2\data\data_241025\\00272_QramProtocol3QTomo.h5'
# data, attrs = prev_data(expt_path, '00009_QramProtocol3QTomo.h5')
saved_files.append(filename)
data, attrs = prev_data(filename, None)
pulse_dict = attrs['pulse_dict']
print('calib_order', attrs['calib_order'])
print('meas_order', attrs['meas_order'])

print(data.keys())
# # data = None
# # style.use('default')
# # qramExpt.analyze(data=data)
# # qramExpt.display(data=data, err=False, saveplot=True)
# # print(data)

cfg = AttrDict(attrs['config'])
init_state = cfg.expt.init_state
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits
print('init_state', init_state)
print('play_pulses', play_pulses)
print('tomo_qubits', tomo_qubits)

np.set_printoptions(suppress=True)
n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)
print('counts_calib')
n_calib = n_calib.astype(np.float64)
for r in range(n_calib.shape[0]):
    n_calib[r,:]/=np.sum(n_calib[r,:])
print(n_calib)
print(attrs['calib_order'])

print(f'thresholds={data["thresholds"].tolist()},')
print(f'angles={data["angles"].tolist()},')
print(f'ge_avgs={data["ge_avgs"].tolist()}',',')
print(f'counts_calib={data["counts_calib"].tolist()}')

In [ ]:
# # counts_calib = data['counts_calib']
# counts_calib=[array([3189,  426, 3176,  519, 1176,  198, 1110,  206]), array([ 580, 3047,  582, 3015,  207, 1148,  221, 1200]), array([1963,  289, 4456,  583,  721,   97, 1661,  230]), array([ 390, 1850,  812, 4372,  130,  683,  269, 1494]), array([1336,  209, 1293,  232, 2954,  473, 2981,  522]), array([ 256, 1341,  226, 1320,  512, 2924,  557, 2864]), array([ 809,  118, 1892,  295, 1810,  262, 4215,  599]), array([ 167,  796,  352, 1830,  325, 1776,  781, 3973])]


# # introduce gaussian noise
# np.random.seed(0)
# noise = 2.0
# n = np.copy([counts_calib[4]])
# print(n, sum(n))
# n0 = correct_readout_err(n, n_conf=counts_calib)
# print('correct readout err', n0)
# print('fix neg counts', fix_neg_counts(n0))

# if noise is not None:
#     for n_psi in n:
#         n_meas = sum(n_psi)
#         new_nlast = -1
#         while new_nlast < 0:
#                 new_n_uptolast = np.random.normal(loc=n_psi[:-1], scale=noise*np.sqrt(n_psi[:-1]))
#                 # new_n_uptolast *= 1 - np.isnan(new_n_uptolast) # 0 out the nans
#                 # print('here', new_n_uptolast)
#                 # preserve original total count per measurement
#                 new_nlast = n_meas - sum(new_n_uptolast)
#                 n_psi[:-1] = np.round(new_n_uptolast)
#                 n_psi[-1] = np.round(new_nlast)

# print(1)
# print(n, sum(n))
# print(type(n))
# print(np.shape(n))
# print(n[0].dtype)
# n = np.array(n, dtype=float)
# n1 = correct_readout_err(n, n_conf=counts_calib)
# print('correct readout err', n1)
# print('fix neg counts', fix_neg_counts(n1))


# # print('n1-n2')
# # print(n1-n2)

# print(3)
# n = [[ 287, 1291,  675, 2954,  256, 1250,  554, 2733]]
# print(n)
# n = correct_readout_err(n, n_conf=counts_calib)
# print(sum(n))
# print(fix_neg_counts(n))

# print(4)
# correct_readout_err(counts_calib, counts_calib)

### Set ideal rho

Get ideal psi for fidelity calculation. Change this depending on what you want for ideal final state.

#### Get ideal final qram state from an arbitrary 2Q input density matrix

In [ ]:
print('init state Q0 (switch) Q1 (input):', init_state)

init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
# if init_state == '|0+1>|0+1>':
#     init_rho = np.array([[(0.23+0j), (0.057+0.233j), (0.18-0.099j), (-0.078+0.08j)], [(0.057-0.233j), (0.287+0j), (-0.016-0.171j), (-0.013+0.114j)], [(0.18+0.099j), (-0.016+0.171j), (0.266+0j), (-0.166+0.121j)], [(-0.078-0.08j), (-0.013-0.114j), (-0.166-0.121j), (0.217+0j)]])
# init_rho = qt.ket2dm((name_to_state_2q('|1>|0>') + name_to_state_2q('|1>|1>') + name_to_state_2q('|0>|0>') + 1j*name_to_state_2q('|0>|1>')).unit())

rho_id = get_qram_qSLR_state_from_rho(init_rho)
# rho_id


# # rho_MLE_ZZ_2Q_filepath = 'S:\Connie\experiments\qramLL_4QR2\data\data_240218\\202402281619_init_rhoMLE_ZZ_2Q_01.npz'
# rho_MLE_ZZ_2Q_filepath = 'S:\Connie\experiments\qramLL_4QR2\data\data_240218\\202402291250_init_rhoMLE_ZZ_2Q_01.npz'

# print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

# rho_MLE_ZZ_dict = dict()
# with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
#     for key in npzfile.keys():
#         rho_MLE_ZZ_dict.update({key:npzfile[key]})
# print(f'rho_MLE_ZZ retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

# init_rho = rho_MLE_ZZ_dict[init_state]
# rho_id = get_qram_qSLR_state_from_rho(init_rho)

In [88]:
# psi_id = qt.tensor(psiZ[1]+psiZ[0], psiZ[0], psiZ[1]+psiZ[0]).unit()
# psi_id = qt.tensor(psiZ[1], psiZ[0], psiZ[1]).unit()
# rho_id = qt.ket2dm(psi_id)
# rho_id = qt.ket2dm(psis['010'] + psis['011']).unit()
rho_id = qt.ket2dm(psis['010']).unit()
# print(rho_id.dims)

### Generate test data

In [ ]:
# rho_id = qt.ket2dm(psi10 + psi11).unit()

# n_tomo = generate_counts(rho_id=rho_id, n_tot=8000, noise=None)
n_tomo_sim = generate_counts(rho_id=rho_id, n_tot=100000, evol_mats=evol_mats, noise=None)
# n_tomo = generate_counts(rho_id=rho_MLE_ZZ, n_tot=100000, evol_mats=evol_mats, noise=None)
# n_tomo = n_tomo_sim
print(n_tomo_sim)

print(meas_order)
print(calib_order)
print('difference')
print(np.round((n_tomo_sim - n_tomo)/100000, 2))

In [ ]:
# n_tomo = generate_counts(rho_id=rho_id, n_tot=8000, noise=None)
# n_tomo = generate_counts(rho_id=rho_id, n_tot=8000, evol_mats=evol_mats, noise=None)
# n_calib = qt.qeye(4).full()
# print(fix_neg_counts(n_tomo))

### Run MLE without ZZ correction

In [ ]:
# methods = 'Nelder-Mead' 'Powell' 'CG' 'BFGS' 'Newton-CG' 'L-BFGS-B' 'TNC' 'COBYLA' 'SLSQP' 'dogleg' 'trust-ncg'
# rho_MLE_base = run_MLE(n=n_tomo, rho_id=rho_id, ZZ_correction=False, method='L-BFGS-B')
rho_MLE_base = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=True,
    correct_neg_counts=True,
    method='analytical',
    ZZ_correction=False,
)

print(rho_MLE_base)

In [ ]:
fid_base = qt.fidelity(qt.Qobj(rho_MLE_base, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
purity_base = np.real(np.trace(rho_MLE_base @ rho_MLE_base))
print(f'Fidelity (no ZZ correction): {fid_base}')
print(f'Purity (no ZZ correction): {purity_base}')
print(np.around(rho_MLE_base, decimals=3))

init_state_title = ''
for c in init_state:
    if c != '>': init_state_title += c
    else: init_state_title += '\\rangle'
tomo_analysis.show_plot_rho_2d(rho_MLE_base, rho_id, title=f'Base MLE (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})\nInit ${init_state_title}$, Play {play_pulses}', savetitle=None, cmax=0.5)

print('Saved data:')
print(*saved_files)

In [ ]:
print()

### Run MLE with ZZ

#### Get evol mats for ZZ

In [16]:
# Recover old config
yaml_cfg = AttrDict(attrs['config'])

NameError: name 'attrs' is not defined

In [18]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [19]:
tomo_qubits = [0, 2, 3]

In [20]:
# evol_mats_path = os.path.join(os.getcwd(), 'evol_mats')
evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"
print('Will save evol mats to path', evol_mats_path)

qA, qB, qC = tomo_qubits
evol_mats_filename = f'evol_mats_{qA}{qB}{qC}.npz'
# evol_mats_filename = f'evol_mats_{qA}{qB}{qC}_old.npz'
print('Will save to filename', evol_mats_filename)

Will save evol mats to path S:\QRAM\qram_4QR2\evol_mats
Will save to filename evol_mats_023.npz


In [ ]:
evol_mats = tomo_analysis.get_evol_mats_from_yaml(
    tomo_qubits=tomo_qubits,
    yaml_cfg=yaml_cfg,
    pulse_dict=pulse_dict,
    cutoffs=[4]*(len(tomo_qubits)),
    soccfg=soc,
    debug=False,
    evol_mats_path=evol_mats_path,
    evol_mats_filename=evol_mats_filename,
)

#### Reload evol mats

In [21]:
qA, qB, qC = tomo_qubits
evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
# evol_mats_file_path = os.path.join(evol_mats_path, f'evol_mats_{qA}{qB}{qC}_test.npz')
print(f'Using evol mats from file {evol_mats_file_path}')

evol_mats = dict()
with np.load(evol_mats_file_path) as npzfile:
    for key in npzfile.keys():
        evol_mats.update({key:npzfile[key]})
# print(evol_mats)

Using evol mats from file S:\QRAM\qram_4QR2\evol_mats\evol_mats_023.npz


#### (Actually) run MLE with ZZ correction

In [80]:
init_state = "|0+1>|0+i>"
print(n_tomo_corrected_dict.keys())
n_tomo_raw = n_tomo_corrected_dict[init_state]
n_calib = None
rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=False,
    method='analytical',
    ZZ_correction=True,
    evol_mats=evol_mats
)

dict_keys(['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>', 'saved_files', 'init_states', 'all_configs', 'allow_pickle'])


 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1178ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1183

In [ ]:
rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=True,
    correct_neg_counts=True,
    method='analytical',
    ZZ_correction=True,
    evol_mats=evol_mats
)

In [81]:
# GET IDEAL RHO FROM DICTIONARY
print('init state to get ideal rho', init_state)
init_rho = rho_MLE_ZZ_dict[init_state]
rho_id = get_qram_qSLR_state_from_rho(init_rho)

init state to get ideal rho |0+1>|0+i>


In [82]:
print(np.around(rho_MLE_ZZ, decimals=3))
fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
print(f'Fidelity (with ZZ correction): {fid_ZZ}')
print(f'Purity (with ZZ correction): {purity_ZZ}')
print(f'Target purity: {purity_id}')
# savetitle = '1_0+1_ideal.png'
# savetitle = 'test.png'
savetitle=None

# if savetitle is None: plt.style.use('default')
# show_plot_rho_2d(rho_id, rho_id, title='MLE with ZZ Correction', savetitle=savetitle) #, cmax=1.0)
print('init state', init_state)
init_state_title = ''
for c in init_state:
    if c != '>': init_state_title += c
    else: init_state_title += '\\rangle'
tomo_analysis.show_plot_rho_2d(rho_MLE_ZZ, rho_id, title=f'MLE with ZZ Correction (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]}), Init ${init_state_title}$, Play {play_pulses}', savetitle=savetitle, cmax=0.5)

if save_data:
    print('Saved data:')
    print(saved_files[-1])

[[ 0.26 +0.j    -0.015+0.j    -0.069+0.239j -0.003-0.j     0.234+0.005j
   0.161+0.189j  0.006+0.003j  0.001-0.005j]
 [-0.015-0.j     0.004+0.j     0.003-0.01j   0.001+0.j    -0.012+0.005j
  -0.013-0.006j  0.002-0.001j -0.001+0.001j]
 [-0.069-0.239j  0.003+0.01j   0.245+0.j    -0.   +0.001j -0.056-0.221j
   0.138-0.2j   -0.001-0.009j -0.003+0.j   ]
 [-0.003+0.j     0.001-0.j    -0.   -0.001j  0.001+0.j     0.   -0.001j
   0.   +0.001j  0.   -0.j     0.   +0.001j]
 [ 0.234-0.005j -0.012-0.005j -0.056+0.221j  0.   +0.001j  0.228+0.j
   0.156+0.184j  0.005-0.j     0.002-0.001j]
 [ 0.161-0.189j -0.013+0.006j  0.138+0.2j    0.   -0.001j  0.156-0.184j
   0.258+0.j     0.003-0.005j  0.002-0.003j]
 [ 0.006-0.003j  0.002+0.001j -0.001+0.009j  0.   +0.j     0.005+0.j
   0.003+0.005j  0.002+0.j    -0.001+0.j   ]
 [ 0.001+0.005j -0.001-0.001j -0.003-0.j     0.   -0.001j  0.002+0.001j
   0.002+0.003j -0.001-0.j     0.001+0.j   ]]
Fidelity (with ZZ correction): 0.43974917517029566
Purity (with ZZ co

NameError: name 'save_data' is not defined

### Optimize over virtual Z gates on ideal rho 

Optimize over virtual Z gates on ideal rho 

In [75]:
# rho_MLE = rho_MLE_base
rho_MLE = rho_MLE_ZZ

In [ ]:
phis = [None, None, None]
phis[0] = np.linspace(0, 360, 30)
phis[1] = np.linspace(0, 360, 30)
phis[2] = np.linspace(0, 360, 30)
# phis[0] = np.linspace(0, 360, 1)
# phis[1] = np.linspace(0, 360, 100)
# phis[2] = np.linspace(0, 360, 100)

rho_MLE_rot, best_phis, best_fid, fids_grid = opt_virtualZ_MLE(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis)
print('Rotation:', best_phis)

In [ ]:
inner_sweep = phis[2]
outer_sweep = phis[1]
y_sweep = outer_sweep
x_sweep = inner_sweep

# vmax = np.max(fids_grid)
vmax = 1.0

# best_phis =  [350.7692307692307, 304.6153846153846, 138.46153846153845]
# best_phis =  [115, 350, 203]
# best_phis = [350.7692307692307, 18.46153846153846, 258.46153846153845]

saveplot = True
plot_filename = filename[:-3]+'.svg'
if saveplot: plt.style.use('dark_background')
best_phi_inds = [np.argmin(np.abs(phis[i]-best_phis[i])) for i in range(3)]

for i_phi0, phi0 in enumerate(phis[0]):
    if i_phi0 != best_phi_inds[0]: continue 
    print(f'phi Q0 {phi0}')
    plt.figure()
    plt.title(f'Fidelities ($\phi_0: ${phi0:0.4})', fontsize=18)
    plt.xlabel(f'$\phi_{tomo_qubits[2]}$', fontsize=18)
    plt.xticks(fontsize=14)
    plt.ylabel(f'$\phi_{tomo_qubits[1]}$', fontsize=18)
    plt.yticks(fontsize=14)
    plt.pcolormesh(y_sweep, x_sweep, fids_grid[i_phi0], cmap='viridis', shading='auto')
    
    plt.plot([best_phis[2]], [best_phis[1]], 'r', marker='*', markersize=15)
    print(f'fid at phi2={best_phis[1]}, phi3={best_phis[2]}:', fids_grid[i_phi0][best_phi_inds[1]][best_phi_inds[2]])
    
    # old_best_phis = [0.0, 94.54545454545455, 349.09090909090907]
    # old_best_phis = [0.0, 196.36363636363635, 341.8181818181818]
    # plt.plot([old_best_phis[2]], [old_best_phis[1]], 'r', marker='o')
    
    divider = make_axes_locatable(plt.gca())
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(cax=cax, ticks=[0, 0.5, vmax])
    # cbar.ax.set_title(f'Fidelity')
    cbar.ax.tick_params(labelsize=12)
    plt.clim(vmin=0, vmax=vmax)
    
    if saveplot:
        plt.savefig(plot_filename, format='svg', bbox_inches='tight', transparent=True)
        print('Saved', plot_filename)
        
    plt.show()
    
# print(f'Difference between phi0[0] and phi0[1]')
# plt.figure()
# plt.title(f'Difference in fidelities ($\phi_0: $0 vs 30)')
# plt.xlabel(f'phis Q{tomo_qubits[2]}')
# plt.ylabel(f'phis Q{tomo_qubits[1]}')
# plt.pcolormesh(y_sweep, x_sweep, fids_grid[0]-fids_grid[1], cmap='viridis', shading='auto')

# # plt.plot([best_phis[2]], [best_phis[1]], 'r', marker='o')

# # old_best_phis = [0.0, 94.54545454545455, 349.09090909090907]
# # old_best_phis = [0.0, 196.36363636363635, 341.8181818181818]
# # plt.plot([old_best_phis[2]], [old_best_phis[1]], 'r', marker='o')

# plt.colorbar()
# # plt.clim(vmin=0, vmax=1)
# plt.show()

In [76]:
# best_phis = [360.0, 360.0, 235.86206896551724]
best_phis = [-4.0, 109.0, 87.0]
# best_phis = old_best_phis

# rho_MLE = rho_MLE_ZZ

z_phi123 = tomo_analysis.z_gate_nq(best_phis)
rho_MLE_rot = (z_phi123*qt.Qobj(rho_MLE, dims=rho_id.dims)*z_phi123.dag()).unit()
fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2

purity_MLE = np.real(np.trace(rho_MLE @ rho_MLE))
purity_MLE_rot = np.real(np.trace(rho_MLE_rot.full() @ rho_MLE_rot.full()))
purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
print(f'Fidelity (after rotation): {fid_rot}')
print(f'Purity (after rotation): {purity_MLE_rot}')
print(f'Target purity: {purity_id}')
print(f'Best rotation: {best_phis}')

# print(np.around(rho_MLE_rot, decimals=3))
savetitle=None
# savetitle=saved_file[:-3]+f'_ZZcorrect_optrot.svg'
tomo_analysis.show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE with ZZ Correction, Phase Optimized (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})', savetitle=savetitle, cmax=0.5)
print('Saved data')
print(saved_files[-1])

Fidelity (after rotation): 0.9669789931413183
Purity (after rotation): 0.9443756454338341
Target purity: 0.9475112788819094
Best rotation: [-4.0, 109.0, 87.0]
Saved data


NameError: name 'saved_files' is not defined

# Loop over 2Q sweep parameters

In [ ]:
# gain_sweep = np.linspace(2000, 25000, 11)
# opt_lens = 1e3*np.array([2.4201237152470587, 1.1987019004033739, 0.7868408797215013, 0.5851160587684981, 0.4716335769416996, 0.3999234927074055, 0.34781134410506404, 0.3089041362621608, 0.2818412962269934, 0.2636215545192442, 0.2512183319921639])
# opt_freqs = [5891.5, 5891.325, 5890.9, 5890.35, 5889.675, 5888.875, 5887.95, 5887.15, 5886.1, 5885.55, 5884.5]


gain_sweep = np.linspace(18000, 32000, 11)
opt_freqs = [5887.35, 5886.616666666667, 5885.883333333333, 5885.616666666667, 5885.116666666667, 5884.383333333333, 5884.116666666667, 5883.616666666667, 5882.883333333333, 5882.383333333333, 5882.583333333333]
opt_lens = [0.30514258121790533, 0.28952631508344284, 0.27703044436306967, 0.2646698856191053, 0.25368901625658447, 0.24735611563570364, 0.23874769989670325, 0.23044507614293083, 0.22816368954972915, 0.22170985331767395, 0.20962837376879714]

save_data = True
sweep2Q_q = 2
qubit = 2
play_pulses=[1,3]

In [ ]:
saved_files = []

init_state = '|0+1>|1>'

opt_phis = [None]

thresholds = None
angles = None
ge_avgs = None
counts_calib = None


for i_gain, gain in enumerate(gain_sweep):
    
    tomo = meas.QramProtocol3QTomoExperiment(
        soccfg=soc,
        path=expt_path,
        config_file=config_path,
    )
    
    span = np.inf
    npts = 1
    if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
    
    tomo.cfg.expt = dict(
        start=span,
        step=0,
        expts=1, # number of different time experiments
        reps=30000, # number of reps per tomo/time step,
        singleshot_reps=30000, # reps per state for singleshot calibration
        post_process='scale', # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
        tomo_3q=True,
        tomo_qubits=tomo_qubits,
        init_state=init_state,
        play_pulses=play_pulses,
        thresholds=thresholds,
        angles=angles,
        ge_avgs=ge_avgs,
        counts_calib=counts_calib,
    )
    tomo.cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q] = int(gain)
    tomo.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q] = opt_lens[i_gain]
    tomo.cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = opt_freqs[i_gain]
    print('gain', tomo.cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q],
          'len', tomo.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q],
          'freq', tomo.cfg.device.qubit.f_EgGf_Q[sweep2Q_q])
    try:
        tomo.go(analyze=False, display=False, progress=False, save=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
    
    if save_data: saved_files.append(tomo.save_data())
    data = tomo.data
    if i_gain == 0:
        thresholds = data['thresholds']
        angles = data['angles']
        ge_avgs = data['ge_avgs']
        counts_calib = data['counts_calib']
    
    print()

In [ ]:
print(saved_files)

In [ ]:
print(saved_files)

### Calculate fidelities with ZZ correction and phase optimization

In [ ]:
apply_ps = True
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

In [ ]:
all_n_tomo = []
all_n_calib = []
all_fids = []
all_fids_rot = []

rho_id = qt.ket2dm(psis['001'] + psis['110']).unit()

for saved_file in saved_files:
    saved_file = saved_file.split('\\')
    # print(saved_file)
    data, attrs = prev_data(expt_path, saved_file[-1])
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)

    all_n_tomo.append(n_tomo_raw)
    all_n_calib.append(n_calib)

    rho_MLE = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_raw,
        n_calib=n_calib,
        correct_readout=True,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=True,
        evol_mats=evol_mats,
    )
    
    # show_plot_rho_2d(rho_MLE_ZZ, rho_id, title=f'MLE with ZZ Correction (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})', savetitle=savetitle, cmax=0.5)
    fid_base = qt.fidelity(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"

    all_fids.append(fid_base)
    print('fid base', fid_base)
    
    rho_MLE_rot, best_phis, fid_rot = opt_virtualZ_MLE(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis)
    all_fids_rot.append(fid_rot)
    tomo_analysis.show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE with ZZ Correction, Phase Optimized (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})', savetitle=None, cmax=0.5)
    print('fid rot', fid_rot)
    


In [ ]:
# print(all_n_tomo)
# print(all_n_calib)
# print(all_fids)
print('fidelity with optimized rotation\n', all_fids_rot)
print('saved files\n', saved_files)

In [ ]:
plt.plot(opt_lens, all_fids_rot, label='Q2 measured fidelity')

# decay_time = 10e3 # ns
# max_fid_times = np.linspace(0, max(opt_lens), 100)
# max_fids = np.exp(-max_fid_times/decay_time)
# plt.plot(max_fid_times, max_fids, '--', label=f'{decay_time/1e3} us decay')

# decay_time = 1e3 # ns
# max_fid_times = np.linspace(0, max(opt_lens), 100)
# max_fids = np.exp(-max_fid_times/decay_time)
# plt.plot(max_fid_times, max_fids, '--', label=f'{decay_time/1e3} us decay')

plt.xlabel(f'Length of 2Q swap in full protocol (ns)')
plt.ylabel(f'Q{sweep2Q_q} Fidelity')
plt.title(f'Q{sweep2Q_q}/Q1 parameter sweep from initial state {init_state}')
plt.legend()
plt.show()